# Load the dataset using deeplake


In [ ]:
!pip install deeplake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 400 kB 15.0 MB/s 
     |████████████████████████████████| 132 kB 67.8 MB/s 
     |████████████████████████████████| 79 kB 10.5 MB/s 
     |████████████████████████████████| 6.7 MB 59.0 MB/s 
     |████████████████████████████████| 79 kB 10.4 MB/s 
     |████████████████████████████████| 10.3 MB 42.0 MB/s 
     |████████████████████████████████| 140 kB 61.0 MB/s 
     |████████████████████████████████| 52 kB 822 kB/s 
     |████████████████████████████████| 132 kB 67.4 MB/s 
     |████████████████████████████████| 127 kB 73.3 MB/s 
  Created wheel for deeplake: filename=deeplake-3.1.4-py3-none-any.whl size=484616 sha256=4d1af663688427a44c1b2fdd6afe329567c561b6ca1cca328c0c990f3bb21801
  Stored in directory: /root/.cache/pip/wheels/cc/7c/08/5cec7180de8f7da04206eedaa259e93f9f0570999b8fba5835
Successfully built deeplake
  Attempting uninstall: urllib3
  

In [ ]:
import deeplake
ds = deeplake.load('hub://activeloop/fer2013-train')
ds_test = deeplake.load('hub://activeloop/fer2013-public-test')

ds.visualize(height=500, width=800)

hub://activeloop/fer2013-train loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/fer2013-train
hub://activeloop/fer2013-public-test loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/fer2013-public-test
 * Serving Flask app "dataset_visualizer" (lazy loading)


 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


# Now build the model

the following structure is just a suggestion and you can do it in your own way.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.activations import relu
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras import layers as Layers
import numpy as np

To simplify the code we can build the residual block separately:

In [ ]:
class ResBlock(Model):
    def __init__(self, channels, stride=1):
        super(ResBlock, self).__init__(name='ResBlock')
        self.flag = (stride != 1)
        self.conv1 = Conv2D(channels, 3, stride, padding='same')
        self.batchnorm1 = BatchNormalization()
        self.conv2 = Conv2D(channels, 3, padding='same')
        self.batchnorm2 = BatchNormalization()
        self.relu = ReLU()
        if self.flag:
            self.batchnorm3 = BatchNormalization()
            self.conv3 = Conv2D(channels, 1, stride)

    def call(self, x):
        y = self.conv1(x)
        y = self.batchnorm1(y)
        y = self.relu(y)
        y = self.conv2(y)
        y = self.batchnorm2(y)
        if self.flag:
            x = self.conv3(x)
            x = self.batchnorm3(x)
        y = Layers.add([x, y])
        y = self.relu(y)
        return y

Now we build the ResNet Model with ResBlocks

In [ ]:
class ResNet34(Model):
    def __init__(self):
        super(ResNet34, self).__init__(name='ResNet34')
        self.conv = Conv2D(64, 7, 2, padding='same')
        self.batchnorm = BatchNormalization()
        self.relu = ReLU()
        self.maxpool = MaxPooling2D(3, 2)

        self.resnet_1_1 = ResBlock(64)
        self.resnet_1_2 = ResBlock(64)
        self.resnet_1_3 = ResBlock(64)

        self.resnet_2_1 = ResBlock(128, 2)
        self.resnet_2_2 = ResBlock(128)
        self.resnet_2_3 = ResBlock(128)
        self.resnet_2_4 = ResBlock(128)

        self.resnet_3_1 = ResBlock(256, 2)
        self.resnet_3_2 = ResBlock(256)
        self.resnet_3_3 = ResBlock(256)
        self.resnet_3_4 = ResBlock(256)
        self.resnet_3_5 = ResBlock(256)
        self.resnet_3_6 = ResBlock(256)

        self.resnet_4_1 = ResBlock(512, 2)
        self.resnet_4_2 = ResBlock(512)
        self.resnet_4_3 = ResBlock(512)

        self.globalpool = GlobalAveragePooling2D()
        self.fc1 = Dense(512, activation='relu')
        self.dp1 = Dropout(0.5)
        self.fc2 = Dense(512, activation='relu')
        self.dp2 = Dropout(0.5)
        self.fcf = Dense(7, activation='softmax')


    def call(self, x):
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.resnet_1_1(x)
        x = self.resnet_1_2(x)
        x = self.resnet_1_3(x)

        x = self.resnet_2_1(x)
        x = self.resnet_2_2(x)
        x = self.resnet_2_3(x)
        x = self.resnet_2_4(x)

        x = self.resnet_3_1(x)
        x = self.resnet_3_2(x)
        x = self.resnet_3_3(x)
        x = self.resnet_3_4(x)
        x = self.resnet_3_5(x)
        x = self.resnet_3_6(x)

        x = self.resnet_4_1(x)
        x = self.resnet_4_2(x)
        x = self.resnet_4_3(x)

        x = self.globalpool(x)
        x = self.fc1(x)
        x = self.dp1(x)
        x = self.fc2(x)
        x = self.dp2(x)
        x = self.fcf(x)
        return x

In [ ]:
model = ResNet34()
model.build(input_shape=(None, 48, 48, 1))
model.summary()

Model: "ResNet34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          multiple                  3200      
                                                                 
 batch_normalization_36 (Bat  multiple                 256       
 chNormalization)                                                
                                                                 
 re_lu_17 (ReLU)             multiple                  0         
                                                                 
 max_pooling2d_1 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 ResBlock (ResBlock)         multiple                  74368     
                                                                 
 ResBlock (ResBlock)         multiple                  743

Now compile the model with a proper optimizer and loss function.

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

then fit your model with enough epochs.

In [ ]:
train_x = tf.expand_dims(ds.images, axis=-1)
train_x = tf.cast(x=train_x,dtype=np.float32)
train_y = tf.keras.utils.to_categorical(ds.labels)

In [ ]:
history = model.fit(train_x, train_y, batch_size=32, validation_split=0.15, epochs=50)

Epoch 1/50
763/763 [==============================] - 37s 44ms/step - loss: 1.8168 - accuracy: 0.2633 - val_loss: 1.7464 - val_accuracy: 0.2909
Epoch 2/50
763/763 [==============================] - 32s 42ms/step - loss: 1.7220 - accuracy: 0.3038 - val_loss: 1.8431 - val_accuracy: 0.2515
Epoch 3/50
763/763 [==============================] - 32s 42ms/step - loss: 1.6232 - accuracy: 0.3602 - val_loss: 1.5619 - val_accuracy: 0.3912
Epoch 4/50
763/763 [==============================] - 31s 41ms/step - loss: 1.5598 - accuracy: 0.3910 - val_loss: 2.2857 - val_accuracy: 0.1785
Epoch 5/50
763/763 [==============================] - 32s 42ms/step - loss: 1.5834 - accuracy: 0.3849 - val_loss: 1.5708 - val_accuracy: 0.3891
Epoch 6/50
763/763 [==============================] - 31s 41ms/step - loss: 1.5363 - accuracy: 0.4061 - val_loss: 1.4392 - val_accuracy: 0.4432
Epoch 7/50
763/763 [==============================] - 31s 41ms/step - loss: 1.5194 - accuracy: 0.4134 - val_loss: 1.6216 - val_accuracy:

In [ ]:
test_x = tf.expand_dims(ds_test.images, axis=-1)
test_x = tf.cast(x=test_x,dtype=np.float32)
test_y = tf.keras.utils.to_categorical(ds_test.labels)

In [ ]:
y_pred = model.evaluate(test_x, test_y)

113/113 [==============================] - 2s 14ms/step - loss: 1.5435 - accuracy: 0.5255
